Vamos a listar las "jotas", es decir, el índice de las coordenadas totales, siguiendo este orden: de la esquina inferior [1,1], ese será j=1, luego j=2 corresponderá a [1,2], j=64 será [1,64] y luego j=65 será [2,1]. Recuerda que esto está ordenado como MATRICES, entonces j=5, [1,5]  es el lugar coordenada x=5 y y=1. Respeta el orden y todo saldrá bien.

In [1]:
ConjuntoDeCoordenadasTotal=Array[]

0-element Array{Array{T,N},1}

Por cierto, estoy SEGURO que esta NO es la forma más inteligente de hacerlo, pero funciona. Ya después optimizamos.

In [2]:
for j=1:64,k=1:64
    push!(ConjuntoDeCoordenadasTotal,[j,k])
end


In [3]:
BceroDura=readdlm("BceroDura.dat");
BceroSuave=readdlm("BceroSuave.dat")
btildedura=readdlm("BtildeDuraCero.dat")
btildesuave=readdlm("BtildeSuaveCero.dat");

In [4]:
BceroSuave[64,64]=393.105 #Convergencia, hay que checar el notebook anterior y ver porque pasa eso...

393.105

In [5]:
BceroDura[64,64]=137.089 #Correccion por convergencia, hay que checar el notebook anterior y ver porque pasa eso...

137.089

In [6]:
addprocs(11)

11-element Array{Int64,1}:
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12

In [63]:
nprocs()

12

Esta parte de abajo debe ser trivialmente paralelizable, ¿no?

In [19]:
jmax=512
xconstante=[64,64]
Kduras=zeros(jmax,jmax);
@time for j=1:jmax
     xj=ConjuntoDeCoordenadasTotal[j]
     for k=1:j
            xk=ConjuntoDeCoordenadasTotal[k]
        for l=1:jmax
            xl=ConjuntoDeCoordenadasTotal[l]
            Kduras[j,k]+=BceroDura[(xk-xl+xconstante)...]*BceroDura[(xj-xl+xconstante)...]
            end
    end
end

159.984415 seconds (1.61 G allocations: 57.125 GB, 4.21% gc time)


In [48]:
length(ConjuntoDeCoordenadasTotal),nprocs()
a=1:348
length(a)

348

In [77]:
@everywhere function sumaalgunasB(LasB::Array, lasX::Array,rango)
    tantas=length(rango)
    tamanho=length(lasX)
    result=zeros(tamanho,tamanho)
    xconstante=[64,64]
    for j=1:tamanho
     xj=lasX[j]
         for k=1:j
            xk=lasX[k]
            aux=0
            for l in rango
                xl=lasX[l]
                aux = LasB[(xk-xl+xconstante)...]*LasB[(xj-xl+xconstante)...]
          end
            result[j,k]=aux
  #  println("hola loco ", KdurasParalel[j,k])
        end
    end
    return result
    
end

In [50]:
@time sumaalgunasB(BceroDura, ConjuntoDeCoordenadasTotal, 1:2);

717.073830 seconds (8.82 G allocations: 331.581 GB, 6.05% gc time)


4096x4096 Array{Float64,2}:
 177.196    0.0      0.0      0.0    …    0.0      0.0      0.0      0.0  
 180.861  184.601    0.0      0.0         0.0      0.0      0.0      0.0  
 184.68   188.499  192.48     0.0         0.0      0.0      0.0      0.0  
 188.663  192.565  196.631  200.872       0.0      0.0      0.0      0.0  
 192.821  196.809  200.965  205.299       0.0      0.0      0.0      0.0  
 197.166  201.244  205.493  209.925  …    0.0      0.0      0.0      0.0  
 201.71   205.881  210.229  214.763       0.0      0.0      0.0      0.0  
 206.467  210.737  215.187  219.828       0.0      0.0      0.0      0.0  
 211.453  215.826  220.384  225.137       0.0      0.0      0.0      0.0  
 216.684  221.166  225.836  230.707       0.0      0.0      0.0      0.0  
 222.18   226.775  231.564  236.558  …    0.0      0.0      0.0      0.0  
 227.96   232.674  237.588  242.712       0.0      0.0      0.0      0.0  
 234.046  238.887  243.931  249.192       0.0      0.0      0.0      0.0

In [78]:
@everywhere function divideArray(lasX::Array)
    procesos=nprocs()
    largo=length(lasX)
    idx=myid()
    splits=[round(Int, s) for s in linspace(0,largo,procesos+1)]
    return splits[idx]+1:splits[idx+1]
end

In [79]:
@everywhere function obtenKParcial(LasB,lasX)
    rangito=divideArray(lasX)
    @show (rangito)
    result= sumaalgunasB(LasB, lasX,rangito)
    return result
end


In [64]:
LasXchiquitas=ConjuntoDeCoordenadasTotal[1:360];
@time obtenKParcial(BceroDura,LasXchiquitas)

rangito = 1:30
  3.405265 seconds (41.00 M allocations: 1.542 GB, 6.45% gc time)


360x360 Array{Float64,2}:
  499.368     0.0       0.0    …     0.0      0.0      0.0      0.0 
  516.829   534.901     0.0          0.0      0.0      0.0      0.0 
  535.541   554.267   574.334        0.0      0.0      0.0      0.0 
  555.64    575.069   595.889        0.0      0.0      0.0      0.0 
  577.286   597.471   619.103        0.0      0.0      0.0      0.0 
  600.66    621.663   644.17   …     0.0      0.0      0.0      0.0 
  625.974   647.862   671.318        0.0      0.0      0.0      0.0 
  653.476   676.326   700.813        0.0      0.0      0.0      0.0 
  683.457   707.356   732.965        0.0      0.0      0.0      0.0 
  716.259   741.304   768.143        0.0      0.0      0.0      0.0 
  752.29    778.595   806.784  …     0.0      0.0      0.0      0.0 
  792.037   819.732   849.41         0.0      0.0      0.0      0.0 
  836.087   865.322   896.651        0.0      0.0      0.0      0.0 
    ⋮                          ⋱                                    
 2271.67

In [97]:
function sacalaKenTrozos(LasB::Array, lasX::Array)
    largo=length(lasX)
    cachos=nprocs()
    aux=SharedArray(Float64,(largo,largo,cachos))
     @sync begin
        for p in procs()
            @async  aux[:,:,p]=remotecall_fetch(p, obtenKParcial, LasB, lasX)
        end
        end #cierra el syncbegin
    result=zeros(largo,largo)
    result=sum(aux,3)
    return result
    #return result
end

sacalaKenTrozos (generic function with 1 method)

In [91]:
@time putin=sacalaKenTrozos(BceroDura,LasXchiquitas);

rangito = 1:30
	From worker 2:	rangito = 31:60
	From worker 3:	rangito = 61:90
	From worker 4:	rangito = 91:120
	From worker 5:	rangito = 121:150
	From worker 7:	rangito = 181:210
	From worker 6:	rangito = 151:180
	From worker 8:	rangito = 211:240
	From worker 9:	rangito = 241:270
	From worker 10:	rangito = 271:300
	From worker 12:	rangito = 331:360
	From worker 11:	rangito = 301:330
  7.312196 seconds (41.03 M allocations: 1.554 GB, 4.52% gc time)


In [92]:
tralala=cumsum(putin,3)

360x360x12 Array{Float64,3}:
[:, :, 1] =
  499.368     0.0       0.0    …     0.0      0.0      0.0      0.0 
  516.829   534.901     0.0          0.0      0.0      0.0      0.0 
  535.541   554.267   574.334        0.0      0.0      0.0      0.0 
  555.64    575.069   595.889        0.0      0.0      0.0      0.0 
  577.286   597.471   619.103        0.0      0.0      0.0      0.0 
  600.66    621.663   644.17   …     0.0      0.0      0.0      0.0 
  625.974   647.862   671.318        0.0      0.0      0.0      0.0 
  653.476   676.326   700.813        0.0      0.0      0.0      0.0 
  683.457   707.356   732.965        0.0      0.0      0.0      0.0 
  716.259   741.304   768.143        0.0      0.0      0.0      0.0 
  752.29    778.595   806.784  …     0.0      0.0      0.0      0.0 
  792.037   819.732   849.41         0.0      0.0      0.0      0.0 
  836.087   865.322   896.651        0.0      0.0      0.0      0.0 
    ⋮                          ⋱                              

In [96]:
?sum

search: sum sum! sumabs summary sumabs2 sumabs! sum_kbn sumabs2! sumaalgunasB



```
sum(A, dims)
```

Sum elements of an array over the given dimensions.

```
sum(itr)
```

Returns the sum of all elements in a collection.

```
sum(f, itr)
```

Sum the results of calling function `f` on each element of `itr`.


In [16]:
@everywhere function obtenerKernelParalell(LasB::Array, lasX::Array, jmax::Int)
    result=SharedArray(Float64,(jmax,jmax))
    @sync @parallel for j=1:jmax
      xj=lasX[j]
         for k=1:jmax
            xk=lasX[k]
            aux=0
          for l=1:jmax
                xl=lasX[l]
                aux = LasB[(xk-xl+xconstante)...]*LasB[(xj-xl+xconstante)...]
          end
            result[j,k]=aux
  #  println("hola loco ", KdurasParalel[j,k])
        end
    end
    result=Array(result)
    return result
end

@everywhere function obtenerKtildeParalell(LasB::Array,LasBTilde::Array, lasX::Array, jmax::Int)
    result=SharedArray(Float64,(jmax,jmax))
    @sync @parallel for j=1:jmax
        xj=lasX[j]
         for k=1:jmax
            xk=lasX[k]
         for l=1:jmax
                xl=lasX[l]
                result[j,k]=200
                unax=xk-xl+xconstante
                unay=xj-xl+xconstante
                if(LasBTilde[unax...]!=0.0)
                    println("hola tu")
                    
                #result[j,k]+= LasB[(xj-xl+xconstante)...]*LasBTilde[(xk-xl+xconstante)...]
                end
        end
  #  println("hola loco ", KdurasParalel[j,k])
        end
    end 
    return result
end
    

In [ ]:
#@time KSuaveParalel=obtenerKernelParalell(BceroSuave, ConjuntoDeCoordenadasTotal,4096);

In [18]:
@time KSuaveParalel2=obtenerKernelParalell(BceroSuave, ConjuntoDeCoordenadasTotal,512);

 41.879294 seconds (262.94 k allocations: 18.312 MB, 0.01% gc time)


In [ ]:
obtenerKtildeParalell(BceroDura,btildedura, ConjuntoDeCoordenadasTotal,128)

In [ ]:
writedlm("KSuavesParalel02.dat", KSuaveParalel)

In [ ]:
jmax=4096
KdurasParalel=SharedArray(Float64, jmax,jmax)

@time @sync @parallel for j=1:jmax
     xj=ConjuntoDeCoordenadasTotal[j]
     for k=1:j
            xk=ConjuntoDeCoordenadasTotal[k]

         for l=1:jmax
            xl=ConjuntoDeCoordenadasTotal[l]
            KdurasParalel[j,k]+= BceroDura[(xk-xl+xconstante)...]*BceroDura[(xj-xl+xconstante)...]
         
       
        end
  #  println("hola loco ", KdurasParalel[j,k])
    end
end 

In [ ]:
writedlm("KDurasParalel.dat", KdurasParalel)

In [ ]:
Kcasero=readdlm("/home/karel/Downloads/KDurasParalel.dat");

In [ ]:
Kcasero-KdurasParalel

In [ ]:
using PyPlot

In [ ]:
ion()
figuru=imshow(KSuaveParalel, origin="upper",interpolation="nearest",cmap="magma") 
cb=colorbar(figuru)

Hay algo raro... me está saliendo Ciclica la integral... ¿tiene sentido?

In [ ]:
figuru=imshow(KTotal, origin="upper",interpolation="nearest",cmap="magma") 
cb=colorbar(figuru)